In [ ]:
import pickle
import os
import sys
import tensorflow as tf
import json

project_root = os.path.abspath('..')
sys.path.insert(0, project_root)

In [ ]:
import json
from methods.config import *
from methods.var import get_VAR_results
from methods.nn import get_NN_results, get_model_name
from methods.data_methods import prepare_model_data
from methods.clean_data import Data_Prep
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, Flatten, Dropout
from keras.models import Sequential
from keras.models import load_model


In [ ]:
for end_year in PERIODS_MAP.values():
    for output_steps in OUTPUT_STEPS:
        for variable in TARGET_VARIABLES:
        
          if end_year not in LONG_HORIZON_COMBINATIONS.get(output_steps, [end_year]):
              continue

          pickle_file_path = os.path.join(TUNING_RESULTS_PATH, f"{end_year}_{output_steps}_{variable}_results.pkl")
          if os.path.isfile(pickle_file_path):
              print(f"Skipping loop for {end_year}_{output_steps}_{variable} because Pickle file already exists.")
              continue

          results = []
          for look_back_year in LOOK_BACK_YEARS:
            for number_of_pca in NUMBER_OF_PCAS:
                    look_back_steps = int(look_back_year * 12)
                    data_prep = Data_Prep(DATA_PATH, TRANSFORM_PATH)
                    data_prep.transform_to_supervised_learning(
                        NA_CUTOFF,
                        [variable],
                        output_steps,
                        start=f"{START_YEAR}-01-01",
                        end=f"{end_year}-01-01",
                    )
                    dataset = data_prep.supervised_dataset
                    full_dataset = dataset["transformed_data"]
                    data = prepare_model_data(
                        window=full_dataset,
                        X_variables=dataset["X_variables"],
                        Y_variables=dataset["Y_variables"],
                        val_steps=VAL_STEPS,
                        look_back=look_back_steps,
                        test_steps=TEST_STEPS,
                        remove_outlier=REMOVE_OUTLIER,
                        number_of_pca=number_of_pca,
                        target_variables=TARGET_VARIABLES,
                    )

                    # Adjust size to match batch
                    data["train_X"] = data["train_X"][
                        len(data["train_X"]) % BATCH_SIZE :
                    ]
                    data["train_Y"] = data["train_Y"][
                        len(data["train_Y"]) % BATCH_SIZE :
                    ]

                    NAME = get_model_name(
                        end_year,
                        variable,
                        FREQUENCY,
                        output_steps,
                        look_back_year,
                        REMOVE_OUTLIER,
                        VAL_YEARS,
                        number_of_pca,
                    )

                    print(f"Evaluating {NAME}")

                    best_model = load_model(os.path.join(TUNING_MODELS_PATH, f"{NAME}.h5"))
                    with open(os.path.join(TUNING_PARAMS_PATH, f"{NAME}.json"), 'r') as f:
                        best_hp = json.load(f)


                    NN_results = get_NN_results(
                        best_model,
                        data,
                        VAL_STEPS,
                        TEST_STEPS,
                        look_back_steps,
                        dataset,
                        BATCH_SIZE,
                        EPOCHS,
                        executions=3,
                        return_model = False
                    )

                    data_prep.transform_to_supervised_learning(
                        NA_CUTOFF,
                        TARGET_VARIABLES,
                        output_steps=output_steps,
                        start=f"{START_YEAR}-01-01",
                        end=f"{end_year}-01-01",
                    )
                    var_dataset = data_prep.supervised_dataset
                    Var_results = get_VAR_results(
                        var_dataset,
                        test_steps=TEST_STEPS,
                        val_steps=VAL_STEPS,
                        output_steps=output_steps,
                    )

                    val_result = {
                        k: v / Var_results["val"]["error"][k]
                        for k, v in NN_results["val"]["error"].items()
                    }
                    val_result["average"] = sum(val_result.values()) / len(
                        val_result.values()
                    )
                    val_result_raw = {
                        k: v / Var_results["val"]["error_raw"][k]
                        for k, v in NN_results["val"]["error_raw"].items()
                    }

                    test_result = {
                        k: v / Var_results["test"]["error"][k]
                        for k, v in NN_results["test"]["error"].items()
                    }
                    test_result["average"] = sum(test_result.values()) / len(
                        test_result.values()
                    )
                    test_result_raw = {
                        k: v / Var_results["test"]["error_raw"][k]
                        for k, v in NN_results["test"]["error_raw"].items()
                    }

                    info = {
                        "val_loss": NN_results["val_loss"],
                        "val_results": val_result,
                        "val_results_raw": val_result_raw,
                        "test_results": test_result,
                        "test_results_raw": test_result_raw,
                        "model_parameters": best_hp,
                        "period": {"start": START_YEAR, "end": end_year},
                        "look_back_years": look_back_year,
                        "output_steps": output_steps,
                        "Var_results": Var_results,
                        "NN_results": NN_results,
                        "variables": variable,
                        "number_of_pca": number_of_pca,
                    }

                    results.append(info)

          with open(pickle_file_path, "wb") as f:
              pickle.dump(results, f)